### 합성곱/풀링 구현하기


In [3]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) # 무작위의 4차원 데이터 생성
print(x.shape) # (채널1개, 세로 28픽셀, 가로 28픽셀)의 데이터가 10개 있다는 의미

print(x[0].shape) # 1번째 데이터의 크기
print(x[1].shape) # 2번째 데이터의 크기

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)


### im2col(image to column)으로 데이터 전개하기

im2col은 입력 데이터를 필터링 하기 좋게 전개하는 함수입니다. 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀝니다. 아래는 그 예시입니다.

![screensh](../../screenshots/im2col.png)

im2col로 입력 데이터를 전개한 다음에는 합성곱 계층의 필터를 1열로 전개하고, 두 행렬의 곱을 계산합니다. <br>
출력한 결과는 2차원 행렬입니다. CNN은 데이터를 4차원 배열로 저장하므로 2차원인 출력 데이터를 4차원으로 변형합니다.

### 합성곱 계층 구현하기

In [9]:
from myfunctions.utils import *

# im2col(input_data, filter_h, filter_w, stride, pad) : 인수의 의미는 차례대로 input_data, 필터의 높이, 필터의 너비, 스트라이드, 패딩 수를 의미합니다.

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride = 1, pad = 0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, 1, 0)
print(col2.shape)

(9, 75)
(90, 75)


In [10]:
class Convolution:
    def __init__(self, W, b, stride = 1, pad = 0):
        self.W = W # 가중치(필터 혹은 커널)
        self.b = b # 편향값
        self.stride = stride # stride
        self.pad = pad # padding
    
    def forward(self, x): # input값을 인자로 받아서
        FN, C, FH, FW = self.W.shape # 필터의 개수, 채널수, 필터의 높이, 필터의 너비
        N, C, H, W = x.shape # input값의 개수, 채널, input값의 높이, input값의 너비
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride) # 출력값의 높이를 공식에 의거해서 계산
        out_W = int(1 + (W + 2*self.pad - FW) / self.stride) # 출력값의 너비를 공식에 의거해서 계산
        
        col = im2col(x, FH, FW, self.stride, self.pad) # img의 컬럼화 함수
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b # 
        
        out = out.reshape(N, out_h, out_W, -1).transpose(0, 3, 1, 2) # transpose의 경우 축의 순서를 인덱스를 기준으로 바꿔준다. 
                                                                     # 원래 N, out_h, out_w, -1 이었던 순서를 N, -1, out_h, out_w 순으로 바꾼다.
        
        return out